# Untitled

## Clean data.

Takes the bone-marrow.arff and creates a tidy tibble

### Load libraries

In [ ]:
#| message: false
library("tidyverse")
library("here")
library("dplyr")
library("ggplot2")

### Clean data

In [ ]:
data_clean_aug <- read_tsv(here('data/03_dat_aug.tsv.gz'))

Rows: 186 Columns: 35
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (23): Recipientgender, Stemcellsource, IIIV, Gendermatch, DonorABO, Reci...
dbl (12): Donorage, Donorage35, CMVstatus, Recipientage, CD34kgx10d6, CD3dCD...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.

### Create graph(s) and visual to explain relation with recipient age, disease type, and BMI

### Recipient age

In [ ]:
 data_clean_aug |>
  select(Recipientgender, Disease, Recipientage, Rbodymass, survival_time, survival_status) |>
  count(Recipientgender, survival_status) |>
   
  ggplot(aes(x = Recipientgender, y = n, fill = factor(survival_status))) +
   
  geom_col(position = "dodge", alpha = 0.5) +
   
  scale_fill_viridis_d() +
  theme_minimal() +
  theme(legend.position = 'bottom')+
  labs( title = 'Survival status among genders',
    x = "Gender",
       y = "Number of survivors/deaths",
       fill = "Survival Status")

Note for me: A **factor** in R is a variable made of **categories**, not
numbers. We use `factor(survival_status)` so ggplot treats **0 (alive)**
and **1 (dead)** as **separate groups** and gives each one a different
color.

In [ ]:
data_clean_aug |>
  ggplot(aes(x = factor(Disease), y = Recipientage, fill = factor(survival_status))) +
  geom_boxplot(alpha = 0.5) + 
   scale_fill_viridis_d() +
  theme_minimal() + 
  labs(
    title = 'Recipient Boxplot',
    x = 'Survival Status', 
    y = 'Age of the recipient',
    fill = 'Survival Status'
  )

Saving 6 x 5 in image

This graphs visualizes the survival rate across disease type and ages.

### Disease type

First I’m doing a plot to see the survival rate against the disease
type, without gender difference.

In [ ]:
data_clean_aug |> count(Disease, survival_status, Recipientgender) |>
  ggplot(aes(x = factor(Disease),
             y = n,
             fill = factor(survival_status))) +
  geom_col(position = 'dodge', alpha = 0.75) + 
    scale_fill_viridis_d() +
  theme_minimal() + 
  labs(
    x = 'Survival Status', 
    y = 'Number of patients',
    fill = 'Survival Status'
  )

Important point –\> none one suffering from lymphoma survived.

### Now I will show the survivors among diseases between genders

In [ ]:
data_clean_aug |>
  mutate(
    Disease = factor(Disease, 
                     levels = sort(unique(Disease)))
  ) |>
  filter(survival_status =='Alive') |>
  count(Disease, Recipientgender) |>
  ggplot(aes(x = Disease, y = n, fill = Recipientgender)) +
  geom_col(position = "dodge", alpha = 0.5) +
  scale_x_discrete(drop = FALSE) + 
    scale_fill_viridis_d()+
  theme_minimal() +
  labs(
    x = "Type of disease",
    y = "Number of survivors",
    fill = "Gender"
  )

### The number of deaths per disease among genders

In [ ]:
data_clean_aug |>
  mutate(
    Disease = factor(Disease, 
                     levels = sort(unique(Disease)))
  ) |>
  filter(survival_status == 'Dead') |>
  count(Disease, Recipientgender) |>
  ggplot(aes(x = Disease, y = n, fill = Recipientgender)) +
  geom_col(position = "dodge", alpha = 0.5) +
  scale_x_discrete(drop = FALSE) +
    scale_fill_viridis_d()+
  theme_minimal() +
  labs(
    x = "Type of disease",
    y = "Number of deaths",
    fill = "Gender"
  )

### Relationship between BMI and survival rate

Graph shows that obese patients had the highest mortality, but also the
highest survival rate among the different Rbodymass index values. this
time we don’t differentiate between women and men, since the indicators
used are from WHO and it foes not differ among genders.

In [ ]:
data_clean_aug |>
  mutate(
    Rbodymass = case_when(
      Rbodymass < 18.5 ~ "Underweight",
      Rbodymass >= 18.5 & Rbodymass < 25 ~ "Normal weight",
      Rbodymass >= 25 & Rbodymass < 30 ~ "Overweight",
      Rbodymass >= 30 ~ "Obese"
    ),
    Rbodymass = fct_relevel(Rbodymass,
                                  'Underweight',
                                  'Normal weight',
                                  'Overweight',
                                  'Obese')
  ) |>
  filter(!is.na(Rbodymass)) |>                      
  count(survival_status, Rbodymass) |>
  group_by(Rbodymass) |>
  
  mutate(Rbodymass_percentage = str_c(round(n / sum(n) * 100, 1), "%")) |>
  
  ggplot(aes(x = factor(Rbodymass),
             y = n,
             fill = factor(survival_status))) +
  
  geom_col(position = 'dodge',
           alpha = 0.5) + 
  
  geom_text(aes(label = Rbodymass_percentage),
            position = position_dodge( width = 1),
            vjust = 0,
            size = 3.5)+
  
  theme_minimal() +
  scale_fill_viridis_d() +
  labs(
    title = "Rbodymass vs Survival rate",
    x = "Rbodymass category",   # this is actually BMI group
    y = "Number of recipients",
    fill = "Survival Status"
  ) +
  theme(legend.position = "bottom")

Saving 6 x 5 in image